In [1]:
import pandas as pd
from pathlib import Path

In [7]:
wb_data = pd.read_csv(
    '/Users/leo/Documents/gpl/eop/data/world_bank_extreme_poverty_rates/API_11_DS2_en_csv_v2_3119.csv'
)

country_list = pd.read_csv(
    '/Users/leo/Documents/gpl/eop/data/world_bank_extreme_poverty_rates/relevant_country_list.csv',
    dtype=str
)

In [8]:

wb_rates = wb_data[wb_data['Indicator Code'] == 'SI.POV.DDAY']
wb_gaps = wb_data[wb_data['Indicator Code'] == 'SI.POV.GAPS']

In [ ]:
for _, row in country_list.iterrows():
    country_data = wb_rates[wb_rates['Country Name'].str.lower() == row['Country'].lower()]
    year = int(row['Year']) if not pd.isna(row['Year']) else None
    if len(country_data) > 0 and year and str(year) in country_data.columns:
        poverty_rate = country_data[str(year)].values[0]
        if pd.isna(poverty_rate):
            # Interpolate missing values
            numeric_data = country_data.loc[:, '1960':'2024'].T
            numeric_data.columns = ['Value']
            numeric_data = numeric_data.dropna().reset_index()
            numeric_data['index'] = numeric_data['index'].astype(int)
            interpolated_value = numeric_data.set_index('index').reindex(range(1960, 2025)).interpolate(method='linear').loc[year, 'Value']
            poverty_rate = interpolated_value if not pd.isna(interpolated_value) else None
        country_list.loc[country_list['Country'] == row['Country'], 'interpolated poverty Rate'] = poverty_rate
        country_list.loc[country_list['Country'] == row['Country'], 'Year of interpolated poverty rate'] = str(year)
    else:
        poverty_rate = None
        print(f'No data found for {row["Country"]}')

No data found for Indonesia
No data found for Chad
No data found for Vietnam


In [ ]:
for _, row in country_list.iterrows():
    country_data = wb_rates[wb_rates['Country Name'].str.lower() == row['Country'].lower()]
    year = int(row['Year']) if not pd.isna(row['Year']) else None
    if len(country_data) > 0 and year and str(year) in country_data.columns:
        poverty_rate = country_data[str(year)].values[0]
        if pd.isna(poverty_rate):
            # Interpolate missing values
            numeric_data = country_data.loc[:, '1960':'2024'].T
            numeric_data.columns = ['Value']
            numeric_data = numeric_data.dropna().reset_index()
            numeric_data['index'] = numeric_data['index'].astype(int)
            interpolated_value = numeric_data.set_index('index').reindex(range(1960, 2025)).interpolate(method='linear').loc[year, 'Value']
            poverty_rate = interpolated_value if not pd.isna(interpolated_value) else None
        country_list.loc[country_list['Country'] == row['Country'], 'interpolated poverty Rate'] = poverty_rate
        country_list.loc[country_list['Country'] == row['Country'], 'Year of interpolated poverty rate'] = str(year)
    else:
        poverty_rate = None
        print(f'No data found for {row["Country"]}')

In [ ]:
country_list.to_csv(
    '/Users/leo/Documents/gpl/eop/data/world_bank_extreme_poverty_rates/relevant_country_list_with_poverty_values.csv',
    index=False
)